In [2]:
import cv2
import pprint
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import time

%matplotlib inline

## 身份证的识别过程
1. 读写图片
2. 调整图片的大小(1000)
3. 新建一个备用的图片
3. 降噪
4. ~~前后背景分离~~
5. 自适应阀值
5. 找出适合的矩形
6. 求出上下左右的边(分别取出一个边)
7. 根据边相交之后的点
8. 做映射变换
9. 写到文件夹里面


In [4]:

strat_time = time.time()
img = '04.jpg'
image_r = cv2.imread(img, cv2.IMREAD_UNCHANGED)
new_img = np.ones_like((image_r),dtype=np.uint8)*255
if image_r.shape[0] > 1000:
    pro_num = 600/image_r.shape[0]
    image_r = cv2.resize(image_r, (0,0), fx=pro_num, fy=pro_num)
image = cv2.pyrMeanShiftFiltering(image_r, 35, 25)
t1 = time.time() 

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 125, 1)

ret, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
outimg, contours, hireachy = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# mask = np.zeros(image.shape[:2],np.uint8)
# bgdModel = np.zeros((1,65),np.float64)
# fgdModel = np.zeros((1,65),np.float64)
# rect = (20,20,image.shape[0]-20, image.shape[1]-20)
# cv2.grabCut(image, mask, rect, bgdModel, fgdModel,5, cv2.GC_INIT_WITH_RECT)
# mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8')
# img = image*mask2[:,:,np.newaxis]
# t2 = time.time()
# cv2.drawContours(image, contours, -1, (0,0,255), 1)

for i, contour in enumerate(contours):
    area = cv2.contourArea(contour)
    if area > 10000:

        approxCurve = cv2.approxPolyDP(contour, 10, True)       
        if approxCurve.shape[0] > 4:
            cv2.drawContours(new_img, contours, i, (0,0,255), 1)
            
        gray_new_img = cv2.cvtColor(new_img, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray_new_img, 100,200) 
#         lines = cv2.HoughLinesP(edges, 1.0, 3*np.pi/180, 100, 5, minLineLength=20, maxLineGap=20)
        lines = cv2.HoughLinesP(edges, 20, 3*np.pi/180, 100 , minLineLength=20, maxLineGap=20)



In [13]:

mask = np.zeros(image.shape[:2],np.uint8)

bgdModel = np.zeros((1,65),np.float64)
fgdModel = np.zeros((1,65),np.float64)
rect = (20,20,image.shape[1]-20, image.shape[0]-20)
cv2.grabCut(image, mask, rect, bgdModel, fgdModel,5, cv2.GC_INIT_WITH_RECT)
mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8')
img = image*mask2[:,:,np.newaxis]
# t2 = time.time()
# cv2.drawContours(image, contours, -1, (0,0,255), 1)
cv2.imshow("mask", mask)
cv2.imshow("img", img)
cv2.imshow("fgdModel", fgdModel)
cv2.imshow("mask2", mask2)


cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
top_lines = []
botton_lines = []
left_lines = []
right_lines = []
lines2D = lines[:,0,:]
for x1,y1,x2,y2 in lines2D[:]:
    if abs(x1 - x2)> abs(y1 - y2):
        if y1 < 250:
            top_lines.append((x1,y1,x2,y2))
    #                     cv2.line(image,(x1,y1),(x2,y2),(255,0,0),1)
        else:
            botton_lines.append((x1,y1,x2,y2))
    #                     cv2.line(image,(x1,y1),(x2,y2),(0,255,0),1)
    else:
        if x1 < 250:
            left_lines.append((x1,y1,x2,y2))
    #                     cv2.line(image,(x1,y1),(x2,y2),(0,0,255),1)
        else:
            right_lines.append((x1,y1,x2,y2))
    #                     cv2.line(image,(x1,y1),(x2,y2),(255,255,0),1)



print(lines.shape)

(52, 1, 4)


In [6]:
def cross_point(line1,line2):#计算交点函数
    x1=line1[0]#取四点坐标
    y1=line1[1] 
    x2=line1[2]
    y2=line1[3]
    print(x1,y1,x2,y2)
    
    x3=line2[0]
    y3=line2[1]
    x4=line2[2]
    y4=line2[3]
    print(x3,y3,x4,y4)

    
    k1=(y2-y1)*1.0/(x2-x1)#计算k1,由于点均为整数，需要进行浮点数转化
    b1=y1*1.0-x1*k1*1.0#整型转浮点型是关键
    if (x4-x3)==0:#L2直线斜率不存在操作
        k2=None
        b2=0
    else:
        k2=(y4-y3)*1.0/(x4-x3)#斜率存在操作
        b2=y3*1.0-x3*k2*1.0
    if k2==None:
        x=x3
    else:
        x=(b2-b1)*1.0/(k1-k2)
    y=k1*x*1.0+b1*1.0
    return [int(x), int(y)]

A = cross_point(top_lines[0], left_lines[0])
B = cross_point(top_lines[0], right_lines[0])
C = cross_point(botton_lines[0], left_lines[0])
D = cross_point(botton_lines[0], right_lines[0])

596 149 692 154
113 409 113 329
596 149 692 154
703 216 703 171
137 523 364 523
113 409 113 329
137 523 364 523
703 216 703 171


In [7]:
corners = np.float32([A,B,C,D])
canvas = np.float32([[0, 0],[800, 0],[0,600],[800,600]])
M = cv2.getPerspectiveTransform(corners, canvas)
result = cv2.warpPerspective(image, M, (800, 600))
# 10.
i = time.localtime().tm_min
cv2.imwrite(f"result_done{i}.jpg",result) 

True

In [8]:
cv2.imshow("result", result)
cv2.imshow("new_img", new_img)
cv2.imshow("gray", gray)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [27]:
image_r.shape

(600, 800, 3)

(1000, 1333, 3)

In [27]:
img = cv2.imread('00.jpg', cv2.IMREAD_GRAYSCALE)
if img.shape[1] > 3000:
    scale = 800/img.shape[0]
    image =cv2.resize(img, (0,0), fx=scale, fy=scale, interpolation=cv2.INTER_NEAREST)
dst = cv2.GaussianBlur(image, (15, 15), 23)
    # 4.自适应图片阀值
gray = cv2.adaptiveThreshold(dst, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 125, 1)
# 5. 二值化图片
ret, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
# 6. 找出最大的矩形
outimg, contours, hireachy = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# 7. 求出上下左右边

for i, contour in enumerate(contours):
    area = cv2.contourArea(contour)
    if area > 10000:

        approxCurve = cv2.approxPolyDP(contour, 10, True)       
    #     if approxCurve.shape[0] > 4:
        cv2.drawContours(image, contours, i, (0,0,255), 1)


cv2.imshow("image", image)
cv2.imshow("graygray", gray)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [25]:
image.shape[0]

800